# Meeting Notebook for August 29, 2025

## Preliminary Notes:
From the August 21 meeting, we established that a transformer model on the aggregated data with a dashboard alone was not sufficient because the amount of data was very small. Instead, we have pivoted to an LSTM/Transformer/DLinear model comparison with the following changes:
- Consider a collection of neighboring counties. Grab the environmental variables from each of the other counties and use that data as extra columns in the feature vector.
- What is the definition of neighboring counties? Well, we want to stretch this definition to three possible interpretations: 
  1. Contiguous connection
  2. All of the central valley counties
  3. All counties surrounding the central valley
- From a recent paper, it seems rats and other rodents are carriers for Coccidioidomycosis. So look into getting rat spray data
- Eventually convert this into a better dashboard (stretch goal)


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import sys 
import os 

sys.path.append(os.path.abspath('..'))

In [3]:
# Set up the data

all_county_df = pd.read_csv("../../data/county_adjacency_list.csv")

In [4]:
all_county_df.head()

,County Name,County GEOID,Neighbor Name,Neighbor GEOID
0,"Autauga County, AL",1001,"Autauga County, AL",1001
1,"Autauga County, AL",1001,"Chilton County, AL",1021
2,"Autauga County, AL",1001,"Dallas County, AL",1047
3,"Autauga County, AL",1001,"Elmore County, AL",1051
4,"Autauga County, AL",1001,"Lowndes County, AL",1085


In [5]:
adj_map = (
  # first remove the repeat of county_name and neighbor_name, as every county is its own
  # neighbor
  all_county_df[all_county_df['County Name'] != all_county_df['Neighbor Name']]

  # you want to groupby the county_name but then select the neighbors
  .groupby('County Name')['Neighbor Name']

  # sort, remove repeats, and remove any NAs
  .apply(lambda s_: sorted(s_.dropna().unique()))

  # Convert to a dict for easy searching
  .to_dict()
)

In [6]:
adj_map["Fresno County, CA"]

['Inyo County, CA',
 'Kings County, CA',
 'Madera County, CA',
 'Merced County, CA',
 'Mono County, CA',
 'Monterey County, CA',
 'San Benito County, CA',
 'Tulare County, CA']

In [7]:
aqi_2008_df = pd.read_csv("../../data/daily_aqi_by_county_2008.csv")
aqi_2008_df.head()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2008-01-04,51,Moderate,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2008-01-07,21,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2008-01-10,20,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2008-01-13,50,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2008-01-16,41,Good,PM2.5,01-003-0010,1


In [8]:
aqi_2008_df['Date'] = pd.to_datetime(aqi_2008_df['Date'])

In [9]:
aqi_2008_df['YearMonth'] = aqi_2008_df['Date'].dt.to_period('M')
aqi_2008_df.head()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,YearMonth
0,Alabama,Baldwin,1,3,2008-01-04,51,Moderate,PM2.5,01-003-0010,1,2008-01
1,Alabama,Baldwin,1,3,2008-01-07,21,Good,PM2.5,01-003-0010,1,2008-01
2,Alabama,Baldwin,1,3,2008-01-10,20,Good,PM2.5,01-003-0010,1,2008-01
3,Alabama,Baldwin,1,3,2008-01-13,50,Good,PM2.5,01-003-0010,1,2008-01
4,Alabama,Baldwin,1,3,2008-01-16,41,Good,PM2.5,01-003-0010,1,2008-01


In [10]:
aqi_monthly_25 = (
  aqi_2008_df[aqi_2008_df['Defining Parameter'] == 'PM2.5']
  .groupby(['county Name', 'YearMonth'])['AQI'].mean()
  .reset_index()
  .groupby('county Name')
  .apply(lambda s: dict(zip(s['YearMonth'].astype(str), s['AQI'])))
  .to_dict()
)

/var/folders/t9/31hxp9pj31j5vk49l32mdc500000gn/T/ipykernel_79204/1299221156.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aqi_2008_df[aqi_2008_df['Defining Parameter'] == 'PM2.5']


In [11]:
aqi_monthly_25['Fresno']

{'2008-01': 80.48148148148148,
 '2008-02': 84.24137931034483,
 '2008-03': 63.5,
 '2008-04': 65.2,
 '2008-05': 64.7,
 '2008-07': 81.0,
 '2008-08': 59.666666666666664,
 '2008-09': 64.16666666666667,
 '2008-10': 68.81818181818181,
 '2008-11': 103.5,
 '2008-12': 93.0}

In [ ]:
aqi_monthly_10["Kern"]

{'2008-01': 101.25,
 '2008-02': 100.0,
 '2008-04': 79.0,
 '2008-05': 94.75,
 '2008-06': 80.0,
 '2008-10': 181.0}

In [14]:
# Go through all the data files and make a massive csv for aqi on monthly scale
from pathlib import Path

data_dir = Path("../../data/")
counties = {"Fresno", "Kings", "Madera", "Merced", "Tulare"}
pollutants = {"PM2.5", "PM10"}              # exact strings as they appear in your files
# If your column is named differently, adjust below:
COL_COUNTY = "county Name"                  # e.g., 'county Name' (match your files)
COL_DATE = "Date"                           # daily date column
COL_POLLUTANT = "Defining Parameter"        # pollutant name column
COL_AQI = "AQI"  

# --- 1) LOAD + CONCAT ALL YEARS ---
files = sorted(data_dir.glob("daily_aqi_by_county_*.csv"))
aqi_daily = pd.concat(
    (pd.read_csv(f, parse_dates=[COL_DATE]) for f in files),
    ignore_index=True
)


In [15]:
aqi_daily

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2008-01-04,51,Moderate,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2008-01-07,21,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2008-01-10,20,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2008-01-13,50,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2008-01-16,41,Good,PM2.5,01-003-0010,1
...,...,...,...,...,...,...,...,...,...,...
2542988,Wyoming,Weston,56,45,2015-12-27,39,Good,Ozone,56-045-0003,2
2542989,Wyoming,Weston,56,45,2015-12-28,39,Good,PM2.5,56-045-0004,2
2542990,Wyoming,Weston,56,45,2015-12-29,44,Good,PM2.5,56-045-0004,2
2542991,Wyoming,Weston,56,45,2015-12-30,46,Good,PM2.5,56-045-0004,2


In [16]:
# --- 2) FILTER TO COUNTIES + POLLUTANTS OF INTEREST ---
aqi_daily = aqi_daily[
    aqi_daily[COL_COUNTY].isin(counties) & aqi_daily[COL_POLLUTANT].isin(pollutants)
].copy()

In [17]:
aqi_daily

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
16732,California,Fresno,6,19,2008-01-01,128,Unhealthy for Sensitive Groups,PM2.5,06-019-0008,7
16733,California,Fresno,6,19,2008-01-02,112,Unhealthy for Sensitive Groups,PM2.5,06-019-0008,5
16734,California,Fresno,6,19,2008-01-03,64,Moderate,PM2.5,06-019-0008,5
16735,California,Fresno,6,19,2008-01-04,55,Moderate,PM2.5,06-019-0008,7
16736,California,Fresno,6,19,2008-01-05,38,Good,PM2.5,06-019-0008,5
...,...,...,...,...,...,...,...,...,...,...
2404933,New York,Kings,36,47,2015-12-27,37,Good,PM2.5,36-047-0118,2
2404934,New York,Kings,36,47,2015-12-28,33,Good,PM2.5,36-047-0052,2
2404935,New York,Kings,36,47,2015-12-29,44,Good,PM2.5,36-047-0118,3
2404936,New York,Kings,36,47,2015-12-30,51,Moderate,PM2.5,36-047-0052,2


In [18]:
# --- 3) MONTHLY AGGREGATION ---
# Make YearMonth and aggregate: use mean for AQI (common convention)
aqi_daily["YearMonth"] = aqi_daily[COL_DATE].dt.to_period("M").astype(str)

In [19]:
aqi_monthly_long = (
    aqi_daily
    .groupby([COL_COUNTY, "YearMonth", COL_POLLUTANT], as_index=False)[COL_AQI]
    .mean()
)

In [20]:
aqi_monthly_long

,county Name,YearMonth,Defining Parameter,AQI
0,Fresno,2008-01,PM2.5,80.481481
1,Fresno,2008-02,PM2.5,84.241379
2,Fresno,2008-03,PM2.5,63.500000
3,Fresno,2008-04,PM2.5,65.200000
4,Fresno,2008-05,PM2.5,64.700000
...,...,...,...,...
485,Tulare,2015-09,PM2.5,47.000000
486,Tulare,2015-10,PM10,72.000000
487,Tulare,2015-10,PM2.5,63.000000
488,Tulare,2015-11,PM2.5,70.518519


In [21]:
# --- 4) PIVOT TO WIDE: columns AQI_PM25, AQI_PM10 ---
aqi_monthly = (
    aqi_monthly_long
    .pivot(index=[COL_COUNTY, "YearMonth"], columns=COL_POLLUTANT, values=COL_AQI)
    .rename(columns={"PM2.5": "AQI_PM25", "PM10": "AQI_PM10"})
    .reset_index()
)

In [22]:
aqi_monthly

Defining Parameter,county Name,YearMonth,AQI_PM10,AQI_PM25
0,Fresno,2008-01,NaN,80.481481
1,Fresno,2008-02,NaN,84.241379
2,Fresno,2008-03,NaN,63.500000
3,Fresno,2008-04,NaN,65.200000
4,Fresno,2008-05,NaN,64.700000
...,...,...,...,...
406,Tulare,2015-08,59.0,NaN
407,Tulare,2015-09,53.0,47.000000
408,Tulare,2015-10,72.0,63.000000
409,Tulare,2015-11,NaN,70.518519


In [23]:
# Pivot again to get wide format: columns = (County, Pollutant)
aqi_wide = (
    aqi_monthly_long
    .pivot(index="YearMonth", columns=[COL_COUNTY, COL_POLLUTANT], values=COL_AQI)
)

# Flatten the MultiIndex columns into "County_Pollutant"
aqi_wide.columns = [f"{county}_{pollutant.replace('.', '')}" for county, pollutant in aqi_wide.columns]

# Optional: rename PM2.5 -> PM25 for cleaner names
aqi_wide.columns = [col.replace("PM2.5", "PM25") for col in aqi_wide.columns]

# Reset index so YearMonth is a normal column
aqi_wide = aqi_wide.reset_index()


In [24]:
aqi_wide

,YearMonth,Fresno_PM25,Fresno_PM10,Kings_PM10,Kings_PM25,Madera_PM25,Madera_PM10,Merced_PM10,Merced_PM25,Tulare_PM25,Tulare_PM10
0,2008-01,80.481481,NaN,82.000000,68.140351,NaN,NaN,31.0,74.333333,79.240000,NaN
1,2008-02,84.241379,NaN,NaN,72.685185,NaN,NaN,NaN,80.142857,94.136364,NaN
2,2008-03,63.500000,NaN,NaN,57.372881,NaN,NaN,NaN,59.142857,65.869565,NaN
3,2008-04,65.200000,NaN,NaN,51.868421,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-05,64.700000,NaN,120.333333,47.264706,NaN,NaN,NaN,NaN,57.428571,NaN
...,...,...,...,...,...,...,...,...,...,...,...
91,2015-08,71.333333,NaN,57.300000,33.580645,56.600000,50.25,NaN,55.285714,NaN,59.0
92,2015-09,96.250000,67.25,69.500000,33.833333,63.000000,59.20,NaN,59.250000,47.000000,53.0
93,2015-10,61.000000,NaN,83.000000,44.280000,54.666667,NaN,NaN,52.769231,63.000000,72.0
94,2015-11,79.461538,NaN,NaN,57.189655,67.222222,NaN,NaN,70.250000,70.518519,NaN


In [25]:
start, end = "2008-10", "2015-12"

aqi_wide["YearMonth"] = pd.PeriodIndex(aqi_wide["YearMonth"], freq="M")
aqi_wide = (
    aqi_wide
    .set_index("YearMonth")
    .reindex(pd.period_range(start, end, freq="M"))  # guarantees a full continuous span
    .rename_axis("YearMonth")
    .reset_index()
)

# Optional: back to 'YYYY-MM' strings
aqi_wide["YearMonth"] = aqi_wide["YearMonth"].astype(str)


In [26]:
aqi_wide

,YearMonth,Fresno_PM25,Fresno_PM10,Kings_PM10,Kings_PM25,Madera_PM25,Madera_PM10,Merced_PM10,Merced_PM25,Tulare_PM25,Tulare_PM10
0,2008-10,68.818182,72.00,87.111111,58.400000,NaN,NaN,58.0,61.200000,57.900000,NaN
1,2008-11,103.500000,NaN,NaN,77.303571,NaN,NaN,NaN,83.777778,106.518519,NaN
2,2008-12,93.000000,NaN,NaN,64.745763,NaN,NaN,37.0,75.300000,87.703704,NaN
3,2009-01,107.709677,NaN,NaN,83.360656,NaN,NaN,31.0,92.363636,100.233333,NaN
4,2009-02,64.777778,NaN,NaN,61.755556,NaN,NaN,NaN,55.166667,70.562500,NaN
...,...,...,...,...,...,...,...,...,...,...,...
82,2015-08,71.333333,NaN,57.300000,33.580645,56.600000,50.25,NaN,55.285714,NaN,59.0
83,2015-09,96.250000,67.25,69.500000,33.833333,63.000000,59.20,NaN,59.250000,47.000000,53.0
84,2015-10,61.000000,NaN,83.000000,44.280000,54.666667,NaN,NaN,52.769231,63.000000,72.0
85,2015-11,79.461538,NaN,NaN,57.189655,67.222222,NaN,NaN,70.250000,70.518519,NaN


In [27]:
output_path = data_dir / "aqi_pm10_pm25_by_county_filtered.csv"
aqi_wide.to_csv(output_path, index=False)


In [28]:
fire_data = pd.read_csv("../../data/CAL_FIRE_Wildland_PublicReport_2000to2018.csv")
fire_data.head()

/var/folders/t9/31hxp9pj31j5vk49l32mdc500000gn/T/ipykernel_79204/3045536928.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  fire_data = pd.read_csv("../../data/CAL_FIRE_Wildland_PublicReport_2000to2018.csv")


,Unit,County,Agency FDID,Incident,Incident Date,Exposure,Fire Name,Incident Type,Property Use,Address/Location,Latitude,Longitude,Cause,Total Loss,Acres Burned,FS Injury,FS Death,Civ Injury,Civ Death
0,TGU,Tehama,52555,5230,08/07/2006,0,NaN,143,961,N I-5 FWY Red Bluff 96080,0.0,0.0,Undetermined,$0,1.00,0,0,0,0
1,TGU,Tehama,52555,5241,08/08/2006,0,NaN,141,961,5 FWY NWRB 96080,0.0,0.0,Equipment,$0,0.25,0,0,0,0
2,TGU,Tehama,52555,5300,08/10/2006,0,NaN,143,400,00015525 OAKRIDGE RD RTR 96021,0.0,0.0,Debris Burning,$0,1.00,0,0,0,0
3,TGU,Tehama,52030,5382,08/13/2006,0,NaN,143,931,99 HWY Richfield 96021,0.0,0.0,Debris Burning,$0,1.00,0,0,0,0
4,TGU,Tehama,52030,5424,08/14/2006,0,NaN,143,931,,0.0,0.0,Undetermined,$0,1.00,0,0,0,0


In [29]:
counties = ["Fresno", "Kings", "Madera", "Merced", "Tulare"]   # your list
start, end = "2008-10", "2015-12"

df = fire_data.copy()

In [30]:
df.head()

,Unit,County,Agency FDID,Incident,Incident Date,Exposure,Fire Name,Incident Type,Property Use,Address/Location,Latitude,Longitude,Cause,Total Loss,Acres Burned,FS Injury,FS Death,Civ Injury,Civ Death
0,TGU,Tehama,52555,5230,08/07/2006,0,NaN,143,961,N I-5 FWY Red Bluff 96080,0.0,0.0,Undetermined,$0,1.00,0,0,0,0
1,TGU,Tehama,52555,5241,08/08/2006,0,NaN,141,961,5 FWY NWRB 96080,0.0,0.0,Equipment,$0,0.25,0,0,0,0
2,TGU,Tehama,52555,5300,08/10/2006,0,NaN,143,400,00015525 OAKRIDGE RD RTR 96021,0.0,0.0,Debris Burning,$0,1.00,0,0,0,0
3,TGU,Tehama,52030,5382,08/13/2006,0,NaN,143,931,99 HWY Richfield 96021,0.0,0.0,Debris Burning,$0,1.00,0,0,0,0
4,TGU,Tehama,52030,5424,08/14/2006,0,NaN,143,931,,0.0,0.0,Undetermined,$0,1.00,0,0,0,0


In [31]:
df.columns = [c.strip() for c in df.columns]

# coerce types
df["County"] = df["County"].str.strip()
df["Incident Date"] = pd.to_datetime(df["Incident Date"], errors="coerce", infer_datetime_format=True)
df["Acres Burned"] = pd.to_numeric(df["Acres Burned"], errors="coerce")

/var/folders/t9/31hxp9pj31j5vk49l32mdc500000gn/T/ipykernel_79204/3039338693.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Incident Date"] = pd.to_datetime(df["Incident Date"], errors="coerce", infer_datetime_format=True)


In [32]:
df.head()

,Unit,County,Agency FDID,Incident,Incident Date,Exposure,Fire Name,Incident Type,Property Use,Address/Location,Latitude,Longitude,Cause,Total Loss,Acres Burned,FS Injury,FS Death,Civ Injury,Civ Death
0,TGU,Tehama,52555,5230,2006-08-07,0,NaN,143,961,N I-5 FWY Red Bluff 96080,0.0,0.0,Undetermined,$0,1.00,0,0,0,0
1,TGU,Tehama,52555,5241,2006-08-08,0,NaN,141,961,5 FWY NWRB 96080,0.0,0.0,Equipment,$0,0.25,0,0,0,0
2,TGU,Tehama,52555,5300,2006-08-10,0,NaN,143,400,00015525 OAKRIDGE RD RTR 96021,0.0,0.0,Debris Burning,$0,1.00,0,0,0,0
3,TGU,Tehama,52030,5382,2006-08-13,0,NaN,143,931,99 HWY Richfield 96021,0.0,0.0,Debris Burning,$0,1.00,0,0,0,0
4,TGU,Tehama,52030,5424,2006-08-14,0,NaN,143,931,,0.0,0.0,Undetermined,$0,1.00,0,0,0,0


In [33]:

# optional: avoid double-counting exposures (keep primary)
if "Exposure" in df.columns:
    df = df[df["Exposure"].astype("Int64").fillna(0) == 0]

# keep only your counties
df = df[df["County"].isin(counties)].copy()

# make YearMonth
df["YearMonth"] = df["Incident Date"].dt.to_period("M")

# monthly aggregation: sum acres per (County, YearMonth)
fire_monthly = (
    df.groupby(["County", "YearMonth"], as_index=False)["Acres Burned"]
      .sum(min_count=1)  # keeps NaN if entirely missing
)

# pivot to wide: columns like Fresno_FIRE_Acres_Burned, Kings_FIRE_Acres_Burned, ...
fire_wide = (
    fire_monthly
    .pivot(index="YearMonth", columns="County", values="Acres Burned")
    .rename(columns=lambda c: f"{c}_FIRE_Acres_Burned")
)

# enforce full continuous month span and fill months with no fires as 0
fire_wide = (
    fire_wide
    .reindex(pd.period_range(start, end, freq="M"))
    .fillna(0.0)
)

# tidy: YearMonth as YYYY-MM string first column
fire_wide = fire_wide.rename_axis("YearMonth").reset_index()
fire_wide["YearMonth"] = fire_wide["YearMonth"].astype(str)

In [34]:
fire_wide.head()

County,YearMonth,Fresno_FIRE_Acres_Burned,Kings_FIRE_Acres_Burned,Madera_FIRE_Acres_Burned,Merced_FIRE_Acres_Burned,Tulare_FIRE_Acres_Burned
0,2008-10,165.91,0.0,16.96,15.68,3.1
1,2008-11,18.30,1.0,5.25,4.53,2.0
2,2008-12,1.00,0.0,0.00,0.00,0.0
3,2009-01,1.00,0.0,0.25,1.11,0.0
4,2009-02,28.10,0.0,0.51,0.01,0.0


In [35]:
fire_wide

County,YearMonth,Fresno_FIRE_Acres_Burned,Kings_FIRE_Acres_Burned,Madera_FIRE_Acres_Burned,Merced_FIRE_Acres_Burned,Tulare_FIRE_Acres_Burned
0,2008-10,165.91,0.0,16.96,15.68,3.10
1,2008-11,18.30,1.0,5.25,4.53,2.00
2,2008-12,1.00,0.0,0.00,0.00,0.00
3,2009-01,1.00,0.0,0.25,1.11,0.00
4,2009-02,28.10,0.0,0.51,0.01,0.00
...,...,...,...,...,...,...
82,2015-08,28.16,0.0,30.48,34.49,12.38
83,2015-09,211.66,0.0,38.16,102.86,12.65
84,2015-10,14.99,0.0,8.02,8.28,38.21
85,2015-11,31.47,0.0,2.60,3.89,0.00


In [37]:
output_name = "wildfire_by_county_processed.csv"
fire_out_path = data_dir / output_name
fire_wide.to_csv(fire_out_path, index=False)

In [69]:
df_fung = pd.read_csv("../../data/Final_FungicideData.csv")

In [70]:
df_fung = df_fung.copy()

In [71]:
fungicide_summed_lbs =(
  df_fung.groupby(["COUNTY_NAME", "Year-Month"])["POUNDS_CHEMICAL_APPLIED"]
  .sum(min_count = 1)
)

In [72]:
fungicide_summed_lbs

COUNTY_NAME  Year-Month
FRESNO       2000-01          0.031250
             2000-02       1862.305293
             2000-03       2726.282281
             2000-04       7492.311048
             2000-05       5687.137561
                              ...     
TULARE       2022-08        117.973703
             2022-09         33.103770
             2022-10        122.812717
             2022-11        926.850587
             2022-12        258.734845
Name: POUNDS_CHEMICAL_APPLIED, Length: 1546, dtype: float64